# 数据集转换

## coseal/Magicoder-Evol-Instruct-110K-sft

In [14]:
from datasets import load_dataset

dataset = load_dataset("coseal/Magicoder-Evol-Instruct-110K-sft")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 100772
    })
})


In [15]:
import pandas as pd

df = pd.DataFrame(dataset["train"])
df

,instruction,response
0,Please amend the subsequent Python script so t...,```python\n# Establish an integer list\narr = ...
1,"i've got this python code from an ocr tool, bu...",There are several issues with your code like f...
2,Create a recursive function in Java that predi...,You can achieve this using a combination of re...
3,Develop a program that uses natural language p...,This task requires writing of a significant vo...
4,I am trying to create a load testing script fo...,The correct approach would depend on the preci...
...,...,...
100767,Create a function that not only detects any ch...,"In Python, we could use the difflib module and..."
100768,Conduct a comprehensive exploratory data analy...,"Sorry, as an AI developed by Open AI, I can't ..."
100769,"I used OCR to get this Python code, but it's f...","Sure, I can help with that. Let me reformat th..."
100770,Write a JavaScript function that will take an ...,"Sure, I'll provide you with a simple example o..."


In [16]:
from metagpt.utils.common import OutputParser

def is_python_code(row):
    # instruction = row.instruction
    response = row.response

    try:
        code = OutputParser.parse_code(response, "python")
        if len(code) > 10:
            return True
    except:
        pass
    return False

df["ispy"] = df.apply(is_python_code, axis=1)
xdf = df[df.ispy][["instruction","response"]].copy().rename(columns={"instruction": "input", "response": "output"})
xdf

2024-03-26 19:23:32.328 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /Users/deryzhou/Downloads/StreamChatPlayground/notebooks


,input,output
0,Please amend the subsequent Python script so t...,```python\n# Establish an integer list\narr = ...
1,"i've got this python code from an ocr tool, bu...",There are several issues with your code like f...
3,Develop a program that uses natural language p...,This task requires writing of a significant vo...
8,Develop a method that not only incorporates a ...,You can implement this task in Python:\n\n```p...
9,Write a Python program that takes a list and a...,Here is a Python program that does just that:\...
...,...,...
100763,Modify and optimize the given erroneous code t...,This erroneous code uses the bubble sort algor...
100764,Utilize the BeautifulSoup package to orchestra...,You can extract the title tag content from the...
100765,Implement a function that can insert multiple ...,Here is the Python implementation:\n\n```pytho...
100767,Create a function that not only detects any ch...,"In Python, we could use the difflib module and..."


In [17]:
xdf.to_json("magicoder.jsonl", orient='records', lines=True)

# 数据收集

## SWE-bench

```bash
$ pip3 install sentencepiece
```

In [1]:
import os, sys
parent_path = os.path.dirname(os.getcwd())
sys.path.append(parent_path)
# SWE_path = os.path.join(os.path.dirname(os.getcwd()), "SWE_bench")
# sys.path.append(SWE_path)

from SWE_bench.inference.make_datasets.create_instance import PROMPT_FUNCTIONS, add_text_inputs, TOKENIZER_FUNCS

PROMPT_FUNCTIONS

2024-03-29 10:54:35,976 WARNING Disabling caching


tokenizer_config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

{'style-2': <function SWE_bench.inference.make_datasets.create_instance.prompt_style_2(instance)>,
 'style-3': <function SWE_bench.inference.make_datasets.create_instance.prompt_style_3(instance)>,
 'full_file_gen': <function SWE_bench.inference.make_datasets.create_instance.full_file_gen(instance)>,
 'style-2-edits-only': <function SWE_bench.inference.make_datasets.create_instance.prompt_style_2_edits_only(instance)>}

In [2]:
from datasets import load_dataset

dataset = load_dataset("princeton-nlp/SWE-bench")
print(dataset)

Generating dev split:   0%|          | 0/225 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2294 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/19008 [00:00<?, ? examples/s]

DatasetDict({
    dev: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 225
    })
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 2294
    })
    train: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 19008
    })
})


In [21]:
import pandas as pd

df = pd.DataFrame(dataset["dev"])
df.iloc[0]

repo                                                        sqlfluff/sqlfluff
instance_id                                           sqlfluff__sqlfluff-4764
base_commit                          a820c139ccbe6d1865d73c4a459945cd69899f8f
patch                       diff --git a/src/sqlfluff/cli/commands.py b/sr...
test_patch                  diff --git a/test/cli/commands_test.py b/test/...
problem_statement           Enable quiet mode/no-verbose in CLI for use in...
hints_text                                                                   
created_at                                               2023-04-16T14:24:42Z
version                                                                   1.4
FAIL_TO_PASS                ["test/cli/commands_test.py::test__cli__fix_mu...
PASS_TO_PASS                ["test/cli/commands_test.py::test__cli__comman...
environment_setup_commit             d19de0ecd16d298f9e3bfb91da122734c40c01e5
Name: 0, dtype: object

In [28]:
from copy import deepcopy

instance = deepcopy(df.iloc[0])
instance["readmes"] = dict()
instance["file_contents"] = dict()
base_text_inputs = PROMPT_FUNCTIONS["style-3"](instance)
print(base_text_inputs)

You will be provided with a partial code base and an issue statement explaining a problem to resolve.
<issue>
Enable quiet mode/no-verbose in CLI for use in pre-commit hook
There seems to be only an option to increase the level of verbosity when using SQLFluff [CLI](https://docs.sqlfluff.com/en/stable/cli.html), not to limit it further.

It would be great to have an option to further limit the amount of prints when running `sqlfluff fix`, especially in combination with deployment using a pre-commit hook. For example, only print the return status and the number of fixes applied, similar to how it is when using `black` in a pre-commit hook:
![image](https://user-images.githubusercontent.com/10177212/140480676-dc98d00b-4383-44f2-bb90-3301a6eedec2.png)

This hides the potentially long list of fixes that are being applied to the SQL files, which can get quite verbose.

</issue>

<code>


</code>

Here is an example of a patch file. It consists of changes to the code base. It specifies the f

In [30]:
tokenizer, tokenizer_func = TOKENIZER_FUNCS["cl100k"]
len(tokenizer_func(base_text_inputs, tokenizer))

738

#  数据集分析

## [princeton-nlp/SWE-bench_Lite](https://huggingface.co/datasets/princeton-nlp/SWE-bench_Lite)

```
instance_id: (str) - A formatted instance identifier, usually as repo_owner__repo_name-PR-number.
patch: (str) - The gold patch, the patch generated by the PR (minus test-related code), that resolved the issue.
repo: (str) - The repository owner/name identifier from GitHub.
base_commit: (str) - The commit hash of the repository representing the HEAD of the repository before the solution PR is applied.
hints_text: (str) - Comments made on the issue prior to the creation of the solution PR’s first commit creation date.
created_at: (str) - The creation date of the pull request.
test_patch: (str) - A test-file patch that was contributed by the solution PR.
problem_statement: (str) - The issue title and body.
version: (str) - Installation version to use for running evaluation.
environment_setup_commit: (str) - commit hash to use for environment setup and installation.
FAIL_TO_PASS: (str) - A json list of strings that represent the set of tests resolved by the PR and tied to the issue resolution.
PASS_TO_PASS: (str) - A json list of strings that represent tests that should pass before and after the PR application.
```

In [3]:
from datasets import load_dataset

dataset = load_dataset("princeton-nlp/SWE-bench_Lite")
dataset["dev"]

Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
    num_rows: 23
})

In [28]:
import json
from loguru import logger
from unidiff import PatchSet

data = dataset["dev"]

results = []
for index in [6]: # range(len(data)):
    instance_id = data[index]["instance_id"]
    logger.info("▶️  Beginning task " + str(index))
    record = data[index]

    issue = record["problem_statement"]
    files = []
    if "patch" in record:
        files = "\n".join(
            [f"- {x.path}" for x in PatchSet(record["patch"]).modified_files]
        )
    test_files = []
    if "test_patch" in record:
        test_patch_obj = PatchSet(record["test_patch"])
        test_files = "\n".join(
            [f"- {x.path}" for x in test_patch_obj.modified_files + test_patch_obj.added_files]
        )

    tests = ""
    if "FAIL_TO_PASS" in record:
        objs = json.loads(record["FAIL_TO_PASS"])
        tests = "\n".join([f"- {x}" for x in objs])

    args = {
        "issue": issue,
        "files": files,
        "test_files": test_files,
        "tests": tests
    }
    results.append(args)
    logger.info(json.dumps(args, indent=2))

print(results[0]["issue"])

2024-04-07 15:58:52.651 | INFO     | __main__:<module>:10 - ▶️  Beginning task 6
2024-04-07 15:58:52.653 | INFO     | __main__:<module>:38 - {
  "issue": "[version 2.20.0] TypeError: 'NoneType' object is not subscriptable\nAfter update from version 2.19.5 to 2.20.0 I got error for code like:\r\n\r\n```python\r\nfrom marshmallow import Schema, fields, validates\r\n\r\n\r\nclass Bar(Schema):\r\n    value = fields.String()\r\n\r\n    @validates('value')  # <- issue here\r\n    def validate_value(self, value):\r\n        pass\r\n\r\n\r\nclass Foo(Schema):\r\n    bar = fields.Nested(Bar)\r\n\r\n\r\nsch = Foo()\r\n\r\nsch.validate({\r\n    'bar': 'invalid',\r\n})\r\n```\r\n\r\n```\r\nTraceback (most recent call last):\r\n  File \"/_/bug_mschema.py\", line 19, in <module>\r\n    'bar': 'invalid',\r\n  File \"/_/env/lib/python3.7/site-packages/marshmallow/schema.py\", line 628, in validate\r\n    _, errors = self._do_load(data, many, partial=partial, postprocess=False)\r\n  File \"/_/env/lib/p

[version 2.20.0] TypeError: 'NoneType' object is not subscriptable
After update from version 2.19.5 to 2.20.0 I got error for code like:

```python
from marshmallow import Schema, fields, validates


class Bar(Schema):
    value = fields.String()

    @validates('value')  # <- issue here
    def validate_value(self, value):
        pass


class Foo(Schema):
    bar = fields.Nested(Bar)


sch = Foo()

sch.validate({
    'bar': 'invalid',
})
```

```
Traceback (most recent call last):
  File "/_/bug_mschema.py", line 19, in <module>
    'bar': 'invalid',
  File "/_/env/lib/python3.7/site-packages/marshmallow/schema.py", line 628, in validate
    _, errors = self._do_load(data, many, partial=partial, postprocess=False)
  File "/_/env/lib/python3.7/site-packages/marshmallow/schema.py", line 670, in _do_load
    index_errors=self.opts.index_errors,
  File "/_/env/lib/python3.7/site-packages/marshmallow/marshalling.py", line 292, in deserialize
    index=(index if index_errors else None)
  F

In [27]:
data[index]

{'repo': 'marshmallow-code/marshmallow',
 'instance_id': 'marshmallow-code__marshmallow-1343',
 'base_commit': '2be2d83a1a9a6d3d9b85804f3ab545cecc409bb0',
 'patch': 'diff --git a/src/marshmallow/schema.py b/src/marshmallow/schema.py\n--- a/src/marshmallow/schema.py\n+++ b/src/marshmallow/schema.py\n@@ -877,7 +877,7 @@ def _invoke_field_validators(self, unmarshal, data, many):\n                 for idx, item in enumerate(data):\n                     try:\n                         value = item[field_obj.attribute or field_name]\n-                    except KeyError:\n+                    except (KeyError, TypeError):\n                         pass\n                     else:\n                         validated_value = unmarshal.call_and_store(\n@@ -892,7 +892,7 @@ def _invoke_field_validators(self, unmarshal, data, many):\n             else:\n                 try:\n                     value = data[field_obj.attribute or field_name]\n-                except KeyError:\n+                ex

In [31]:
import yaml

config_data = """system_template: |-
  SETTING: You are an autonomous programmer, and you're working directly in the command line with a special interface.

  The special interface consists of a file editor that shows you {WINDOW} lines of a file at a time.
  In addition to typical bash commands, you can also use the following commands to help you navigate and edit files.

  COMMANDS:
  {command_docs}

  Please note that THE EDIT COMMAND REQUIRES PROPER INDENTATION. 
  If you'd like to add the line '        print(x)' you must fully write that out, with all those spaces before the code! Indentation is important and code that is not indented correctly will fail and require fixing before it can be run.

  RESPONSE FORMAT:
  Your shell prompt is formatted as follows:
  (Open file: <path>) <cwd> $

  You need to format your output using two fields; discussion and command.
  Your output should always include _one_ discussion and _one_ command field EXACTLY as in the following example:
  DISCUSSION
  First I'll start by using ls to see what files are in the current directory. Then maybe we can look at some relevant files to see what they look like.
  ```
  ls -a
  ```

  You should only include a *SINGLE* command in the command section and then wait for a response from the shell before continuing with more discussion and commands. Everything you include in the DISCUSSION section will be saved for future reference.
  If you'd like to issue two commands at once, PLEASE DO NOT DO THAT! Please instead first submit just the first command, and then after receiving a response you'll be able to issue the second command. 
  You're free to use any other bash commands you want (e.g. find, grep, cat, ls, cd) in addition to the special commands listed above.
  However, the environment does NOT support interactive session commands (e.g. python, vim), so please do not invoke them.
instance_template: |-
  We're currently solving the following issue within our repository. Here's the issue text:
  ISSUE:
  {issue}

  INSTRUCTIONS:
  Now, you're going to solve this issue on your own. Your terminal session has started and you're in the repository's root directory. You can use any bash commands or the special interface to help you. Edit all the files you need to and run any checks or tests that you want. 
  Remember, YOU CAN ONLY ENTER ONE COMMAND AT A TIME. You should always wait for feedback after every command. 
  When you're satisfied with all of the changes you've made, you can submit your changes to the code base by simply running the submit command.
  Note however that you cannot use any interactive session commands (e.g. python, vim) in this environment, but you can write scripts and run them. E.g. you can write a python script and then run it with `python <script_name>.py`.

  NOTE ABOUT THE EDIT COMMAND: Indentation really matters! When editing a file, make sure to insert appropriate indentation before each line! 

  IMPORTANT TIPS:
  1. Always start by trying to replicate the bug that the issues discusses. 
     If the issue includes code for reproducing the bug, we recommend that you re-implement that in your environment, and run it to make sure you can reproduce the bug.
     Then start trying to fix it.
     When you think you've fixed the bug, re-run the bug reproduction script to make sure that the bug has indeed been fixed.
     
     If the bug reproduction script does not print anything when it succesfully runs, we recommend adding a print("Script completed successfully, no errors.") command at the end of the file,
     so that you can be sure that the script indeed ran fine all the way through. 

  2. If you run a command and it doesn't work, try running a different command. A command that did not work once will not work the second time unless you modify it!

  3. If you open a file and need to get to an area around a specific line that is not in the first 100 lines, say line 583, don't just use the scroll_down command multiple times. Instead, use the goto 583 command. It's much quicker. 
     
  4. If the bug reproduction script requires inputting/reading a specific file, such as buggy-input.png, and you'd like to understand how to input that file, conduct a search in the existing repo code, to see whether someone else has already done that. Do this by running the command: find_file "buggy-input.png" If that doensn't work, use the linux 'find' command. 

  5. Always make sure to look at the currently open file and the current working directory (which appears right after the currently open file). The currently open file might be in a different directory than the working directory! Note that some commands, such as 'create', open files, so they might change the current  open file.

  6. When editing files, it is easy to accidentally specify a wrong line number or to write code with incorrect indentation. Always check the code after you issue an edit to make sure that it reflects what you wanted to accomplish. If it didn't, issue another command to fix it.
     

  (Open file: {open_file})
  (Current directory: {working_dir})
  bash-$
next_step_template: |-
  {observation}
  (Open file: {open_file})
  (Current directory: {working_dir})
  bash-$
next_step_no_output_template: |-
  Your command ran successfully and did not produce any output.
  (Open file: {open_file})
  (Current directory: {working_dir})
  bash-$
demonstration_template: |
  Here is a demonstration of how to correctly accomplish this task.
  It is included to show you how to correctly use the interface.
  You do not need to follow exactly what is done in the demonstration.
  --- DEMONSTRATION ---
  {demonstration}
  --- END OF DEMONSTRATION ---
state_command:
  name: state
  code: |
    state() {
      local working_dir="$PWD";
      if [ -z $CURRENT_FILE ]; then
          echo '{"open_file": "n/a", "working_dir": "'$working_dir'"}';
      else
          echo '{"open_file": "'$(realpath $CURRENT_FILE)'", "working_dir": "'$working_dir'"}';
      fi
    };
parse_function: ThoughtActionParser
env_variables:
  WINDOW: 100
  OVERLAP: 2
  CURRENT_LINE: 0
  CURRENT_FILE: ''
  SEARCH_RESULTS: ()
  SEARCH_FILES: ()
  SEARCH_INDEX: 0
command_files:
- config/commands/defaults.sh
- config/commands/search.sh
- config/commands/edit_linting.sh
- config/commands/_split_string.py
parse_command: ParseCommandDetailed
history_processor: DefaultHistoryProcessor
demonstrations:
- trajectories/demonstrations/replay__marshmallow-code__marshmallow-1867__default_sys-env_window100__t-0.20__p-0.95__c-2.00__install-1/marshmallow-code__marshmallow-1867.traj"""


cfg = yaml.safe_load(config_data)
cfg

{'system_template': "SETTING: You are an autonomous programmer, and you're working directly in the command line with a special interface.\n\nThe special interface consists of a file editor that shows you {WINDOW} lines of a file at a time.\nIn addition to typical bash commands, you can also use the following commands to help you navigate and edit files.\n\nCOMMANDS:\n{command_docs}\n\nPlease note that THE EDIT COMMAND REQUIRES PROPER INDENTATION. \nIf you'd like to add the line '        print(x)' you must fully write that out, with all those spaces before the code! Indentation is important and code that is not indented correctly will fail and require fixing before it can be run.\n\nRESPONSE FORMAT:\nYour shell prompt is formatted as follows:\n(Open file: <path>) <cwd> $\n\nYou need to format your output using two fields; discussion and command.\nYour output should always include _one_ discussion and _one_ command field EXACTLY as in the following example:\nDISCUSSION\nFirst I'll start 

In [39]:
args = {
    "command_docs": cfg.get('command_docs', None),
    **cfg['env_variables'],
}
print(json.dumps(args, indent=2))
print(cfg['system_template'])#.format(**args))

{
  "command_docs": null,
  "WINDOW": 100,
  "OVERLAP": 2,
  "CURRENT_LINE": 0,
  "CURRENT_FILE": "",
  "SEARCH_RESULTS": "()",
  "SEARCH_FILES": "()",
  "SEARCH_INDEX": 0
}
SETTING: You are an autonomous programmer, and you're working directly in the command line with a special interface.

The special interface consists of a file editor that shows you {WINDOW} lines of a file at a time.
In addition to typical bash commands, you can also use the following commands to help you navigate and edit files.

COMMANDS:
{command_docs}

Please note that THE EDIT COMMAND REQUIRES PROPER INDENTATION. 
If you'd like to add the line '        print(x)' you must fully write that out, with all those spaces before the code! Indentation is important and code that is not indented correctly will fail and require fixing before it can be run.

RESPONSE FORMAT:
Your shell prompt is formatted as follows:
(Open file: <path>) <cwd> $

You need to format your output using two fields; discussion and command.
Your 